# Google Colab Setup for Streamlit Client

**Important:** This notebook only runs the Streamlit client.


- You must run your FastAPI backend (uvicorn) on your own machine or a public server.
- Start ngrok on the backend machine and copy the HTTPS URL.
- Update the API_URL in orchestrator_streamlit_client.py to match your ngrok URL.
- Do NOT try to run the FastAPI backend in Colab.
- If you get a 405 error, double-check the backend is running and the URL is correct.

# Run Streamlit App with GPU in Google Colab

This notebook will:
- Clone your project from GitHub
- Install all requirements
- Enable GPU support
- Launch Streamlit using Colab's GPU
- Expose the Streamlit UI via ngrok

In [ ]:
# 1. Check GPU availability
import torch
if torch.cuda.is_available():
    print('GPU is available:', torch.cuda.get_device_name(0))
else:
    print('No GPU detected. Go to Runtime > Change runtime type > select GPU.')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:

%cd /content
!rm -rf AIDrivenSummarizationProjectManagement
# 2. Clone your project repository
!git clone https://github.com/NaveenPalisetti/AIDrivenMeetingSummaryProjectRiskDetection.git
%cd AIDrivenMeetingSummaryProjectRiskDetection

In [ ]:
# 3. Install requirements (including GPU-enabled torch if needed)
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install -r requirements.txt --quiet
!pip install streamlit pyngrok --quiet

In [ ]:
# 4. Set up ngrok (replace with your own authtoken if needed)
!ngrok config add-authtoken 37SZRBmhzpCfMAms5WnqAUVpttn_5xWd1fbo38pEPMYw1YCn

In [ ]:
# 1. Kill old processes
!pkill streamlit
!pkill ngrok
from pyngrok import ngrok
ngrok.kill()
!fuser -k 8000/tcp

In [ ]:
!pip install bitsandbytes

In [ ]:
!nvidia-smi

In [ ]:
# 5. Launch Streamlit app with ngrok tunnel
import os
from pyngrok import ngrok
import threading
import time

# Kill previous tunnels if any
ngrok.kill()

# Start Streamlit in a background thread
streamlit_thread = threading.Thread(target=lambda: os.system('streamlit run orchestrator_streamlit_client.py --server.port 8501 --server.headless true'))
streamlit_thread.start()

# Wait longer to ensure Streamlit is up before starting ngrok
print('Waiting 20 seconds for Streamlit to start...')
time.sleep(20)

# Start ngrok tunnel for Streamlit (port 8501)
public_url = ngrok.connect(8501, bind_tls=True)
print('Streamlit public URL:', public_url)

In [ ]:
# Optional: Copy Mistral model from Google Drive to local disk for faster loading
import shutil, os
src = '/content/drive/MyDrive/Dissertation/Project/models/mistral-7B-Instruct-v0.2'
dst = '/content/mistral-7B-Instruct-v0.2'
if not os.path.exists(dst):
    print('Copying Mistral model from Drive to local disk...')
    shutil.copytree(src, dst)
else:
    print('Local copy of Mistral model already exists.')

In [ ]:
# 4b. Start FastAPI backend in background with port check
!pip install uvicorn --quiet
!uvicorn mcp.server.mcp_api:app --host 0.0.0.0 --port 8000 --reload &
import time, socket
print('Waiting 10 seconds for FastAPI backend to start...')
time.sleep(10)
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
result = sock.connect_ex(('localhost', 8000))
print('Port 8000 status:', 'open' if result == 0 else 'closed')
sock.close()